In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys
sys.path.append('../')
import re
import torch
from utils.utils import * 
from tqdm import tqdm
import seaborn as sns
from sklearn.model_selection import train_test_split
set_seed(42)

In [ ]:
def extract_number(filename):
    match = re.search(r'output_(\d+)\.npy', filename)
    if match:
        return int(match.group(1))
    else:
        return float('inf')

#  Load the dataset

In [ ]:
N_atoms = 8192
output_dir_short = '../data/output_atoms_8192_steps_2000000'
# output_dir_long = '../data/output_atoms_8192_steps_2000000_near_equilibrium'

In [ ]:
def preprocess(output_dir):
    for folder in tqdm(os.listdir(output_dir)):
        T = int(folder.split('_')[1])
        folder_path = os.path.join(output_dir, folder)
        log_path = os.path.join(folder_path, 'chemical_order.csv')
        df = pd.read_csv(log_path)
        
        # ========= macroscopic data =========
        macro_val = np.array([
            df['delta_NbNb'].values,
            df['delta_NbMo'].values,
            df['delta_NbTa'].values,
            df['delta_MoMo'].values,
            df['delta_MoTa'].values,
            df['delta_TaTa'].values
        ]).T

        # ========= T,steps,time data =========
        time_path = os.path.join(folder_path, 'log.csv')
        df_time = pd.read_csv(time_path)
        time = df_time['time'].values
        time = np.insert(time, 0, 0.0)  # Insert initial time step
        T_state = np.ones(macro_val.shape[0], dtype=np.int32) * int(T)
        step = df['step'].values

        config_path = os.path.join(folder_path, 'config_data')
        file_path = [f for f in os.listdir(config_path) if f.endswith('.npy')]
        file_path = sorted(file_path, key=extract_number)
        
        # ========= microscopic data =========
        # micro_val = []
        # for file in file_path:
        #     file_full_path = os.path.join(config_path, file)
        #     grid = np.load(file_full_path)
        #     micro_val.append(grid)
        # micro_val = np.array(micro_val)

        # ========= partial macroscopic data =========
        # idx_partial = np.random.randint(0, 8, size=(macro_val.shape[0]-1,))
        # z0_partial = []
        # z1_partial = []
        # for j in range(idx_partial.shape[0]):
        #     idx = idx_partial[j]
        #     partial_grid_0 = micro_val[j].reshape(2, 16, 2, 16, 2, 16).transpose(0, 2, 4, 1, 3, 5).reshape(-1, 16, 16, 16)[idx]
        #     partial_grid_1 = micro_val[j+1].reshape(2, 16, 2, 16, 2, 16).transpose(0, 2, 4, 1, 3, 5).reshape(-1, 16, 16, 16)[idx]

        #     order_0 = cal_local_chemical_order(partial_grid_0)
        #     order_1 = cal_local_chemical_order(partial_grid_1)
        #     z0_partial.append(order_0)
        #     z1_partial.append(order_1)

        # z0_partial = np.array(z0_partial)
        # z1_partial = np.array(z1_partial)
        # macro_val_partial = macro_val[:-1] + (z1_partial - z0_partial)


        # ========= save data =========
        # np.save(os.path.join(folder_path, 'micro_val.npy'), micro_val)
        np.save(os.path.join(folder_path, 'macro_val.npy'), macro_val)
        np.save(os.path.join(folder_path, 'time.npy'), time)
        np.save(os.path.join(folder_path, 'T_state.npy'), T_state)
        np.save(os.path.join(folder_path, 'step.npy'), step)
        # np.save(os.path.join(folder_path, 'macro_val_partial.npy'), macro_val_partial)

In [ ]:
preprocess(output_dir_short)
# preprocess(output_dir_long)

In [ ]:
# micro_state = [] 
macro_state = []
macro_state_partial = []
T_state = []
time_state = []
step_state = []

# for output_dir in [output_dir_short, output_dir_long]:
for output_dir in [output_dir_short]:
    for folder in tqdm(os.listdir(output_dir)):
        T = int(folder.split('_')[1])
        folder_path = os.path.join(output_dir, folder)

        # micro_state.append(np.load(os.path.join(folder_path, 'micro_val.npy')))
        macro_state.append(np.load(os.path.join(folder_path, 'macro_val.npy')))
        macro_state_partial.append(np.load(os.path.join(folder_path, 'macro_val_partial.npy')))
        time_state.append(np.load(os.path.join(folder_path, 'time.npy')))
        T_state.append(np.load(os.path.join(folder_path, 'T_state.npy')))
        step_state.append(np.load(os.path.join(folder_path, 'step.npy')))


# micro_state = np.stack(micro_state, axis=0)
macro_state = np.stack(macro_state, axis=0)
macro_state_partial = np.stack(macro_state_partial, axis=0)
time_state = np.stack(time_state, axis=0)
T_state = np.stack(T_state, axis=0)
step_state = np.stack(step_state, axis=0)

In [ ]:
T = 2000
indices = np.where(T_state[:, 0] == T)[0]
fig = plt.figure(figsize=(40, 6))
for i in range(6):
    axes = fig.add_subplot(1, 6, i+1)
    for j in indices:
        axes.plot(time_state[j], macro_state[j, :, i])
    axes.set_xlabel('Step')
    axes.set_ylabel(f'Delta_{i}')
    axes.hlines(0, 0, np.max(time_state[indices]), colors='black', linestyles='dashed', linewidth=1)
    axes.set_ylim(-3.5, 3.5)
plt.title(f'T = {T} K')
plt.tight_layout()

In [ ]:
T = 2000
indices = np.where(T_state[:, 0] == T)[0]
fig = plt.figure(figsize=(40, 6))
for i in range(6):
    axes = fig.add_subplot(1, 6, i+1)
    for j in indices:
        axes.plot(time_state[j, :-1], macro_state_partial[j, :, i])
    axes.set_xlabel('Step')
    axes.set_ylabel(f'Delta_{i}')
    axes.hlines(0, 0, np.max(time_state[j]), colors='black', linestyles='dashed', linewidth=1)
    axes.set_ylim(-3.5, 3.5)
plt.title(f'T = {T} K')
plt.tight_layout()

In [ ]:
final_time = {}
T_unique = np.unique(T_state[:, 0])
for T in T_unique:
    idx = np.where(T_state[:, 0] == T)[0]
    steps = np.unique(step_state[idx, -1])
    final_time[T.item()] = time_state[idx, -1].max().item() 

In [ ]:
T = np.array(list(final_time.keys()), dtype=float)      # e.g. [200,300,…]
t = np.array(list(final_time.values()), dtype=float)    # corresponding final times

# 2) prepare for linear fit:  y = ln t, x = 1/T
# x_high = 1.0 / T[9:]
# y_high = np.log(t[9:])

# x_low = 1.0 / T[:9]
# y_low = np.log(t[:9])

x = 1.0 / T
y = np.log(t)

# 3) do a 1st‐order polyfit: y ≈ m*x + b
# m_high, b_high = np.polyfit(x_high, y_high, 1)
# A_high = np.exp(b_high)                # prefactor
# Ea_over_kB_high = m_high               # slope = Eₐ/k_B

# print(f"Arrhenius fit for high temperature: t = {A_high:.3e} · exp({Ea_over_kB_high:.1f}/T)")

# m_low, b_low = np.polyfit(x_low, y_low, 1)
# A_low = np.exp(b_low)                # prefactor
# Ea_over_kB_low = m_low               # slope = Eₐ/k_B
# print(f"Arrhenius fit for low temperature: t = {A_low:.3e} · exp({Ea_over_kB_low:.1f}/T)")

m, b = np.polyfit(x, y, 1)
A = np.exp(b)                # prefactor
Ea_over_kB = m               # slope = Eₐ/k_B
print(f"Overall Arrhenius fit: t = {A:.3e} · exp({Ea_over_kB:.1f}/T)")  


# 4) compute fitted curve
# t_fit_high = A_high * np.exp(Ea_over_kB_high / T[9:])
# t_fit_low = A_low * np.exp(Ea_over_kB_low / T[:9])
t_fit = A * np.exp(Ea_over_kB / T)

# 5) plot data & fit
plt.figure(figsize=(8,5))
plt.scatter(T, t, label="data", color="C0")
# plt.plot(T[9:], t_fit_high, label="Arrhenius fit (high T)", color="C1")
# plt.plot(T[:9], t_fit_low, label="Arrhenius fit (low T)", color="C2")
plt.plot(T, t_fit, label="Overall Arrhenius fit", color="C3")
plt.yscale("log")
plt.xlabel("Temperature (K)")
plt.ylabel("Final time (ps)")
plt.title("Arrhenius fit of final time")
plt.legend()
plt.grid(True, which="both", ls="--", alpha=0.5)
plt.show()

In [ ]:
# def scale_function(T):
#     # Parameters for different temperature ranges
#     # High temperature (>= 1400K)
#     A_high = 4.089e-08
#     Ea_over_kB_high = 17950.7
    
#     # Low temperature (< 1400K)
#     A_low = 3.464e-06  
#     Ea_over_kB_low = 15784.7 

#     # Create boolean masks for temperature ranges
#     high_temp_mask = T >= 1400
#     low_temp_mask = T < 1400
    
#     # Initialize result array
#     result = np.zeros_like(T, dtype=float)
    
#     # Calculate scaling for high temperatures
#     if np.any(high_temp_mask):
#         result[high_temp_mask] = np.exp(-np.log(A_high) - Ea_over_kB_high/T[high_temp_mask])
    
#     # Calculate scaling for low temperatures
#     if np.any(low_temp_mask):
#         result[low_temp_mask] = np.exp(-np.log(A_low) - Ea_over_kB_low/T[low_temp_mask])
    
#     return result

In [ ]:
def scale_function(T):
    # Parameters for different temperature ranges
    A = 5.088e-09
    Ea_over_kB = 17955.5

    result = np.exp(-np.log(A) - Ea_over_kB / T) 
    return result

In [ ]:
scale_value = scale_function(T_state)
time_state_scaled = time_state * scale_value

In [ ]:
final_time_scaled = {}
for T in T_unique:
    idx = np.where(T_state[:, 0] == T)[0]
    final_time_scaled[T.item()] = time_state_scaled[idx, -1].max().item()

In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(final_time_scaled.keys(), final_time_scaled.values(), label="data", color="C0")
plt.xlabel("Temperature (K)")
plt.ylabel("Final time (ps)")
plt.legend()
plt.grid(True, which="both", ls="--", alpha=0.5)
plt.show()

In [ ]:
macro_state.shape

In [ ]:
train_idx, val_idx = train_test_split(np.arange(macro_state.shape[0]), test_size=0.4)
train_idx.shape, val_idx.shape

In [ ]:
macro_state = torch.tensor(macro_state, dtype=torch.float32)
macro_state_partial = torch.tensor(macro_state_partial, dtype=torch.float32)
T_state = torch.tensor(T_state, dtype=torch.int32)
time_state = torch.tensor(time_state, dtype=torch.float32)
time_state_scaled = torch.tensor(time_state_scaled, dtype=torch.float32)
step_state = torch.tensor(step_state, dtype=torch.int32)


In [ ]:
# save the data
save_dir = f'../data/atoms_{N_atoms}'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# ========= save trainingdata =========
# torch.save(micro_state[train_idx], f'{save_dir}/micro_state.pt')
torch.save(macro_state[train_idx], f'{save_dir}/macro_state.pt')
torch.save(T_state[train_idx], f'{save_dir}/T_state.pt')
torch.save(time_state[train_idx], f'{save_dir}/time_state.pt')
torch.save(time_state_scaled[train_idx], f'{save_dir}/time_state_scaled.pt')

 # ========= save partial data =========
# save the partial macro state and idx_partial
# torch.save(idx_partial[train_idx], f'../data/atoms_{N_atoms}/idx_partial.pt')
torch.save(macro_state_partial[train_idx], f'../data/atoms_{N_atoms}/macro_state_partial.pt')

# ========= save validation data =========
# torch.save(micro_state[val_idx], f'{save_dir}/micro_state_val.pt')
torch.save(macro_state[val_idx], f'{save_dir}/macro_state_val.pt')
torch.save(T_state[val_idx], f'{save_dir}/T_state_val.pt')
torch.save(time_state[val_idx], f'{save_dir}/time_state_val.pt')
torch.save(time_state_scaled[val_idx], f'{save_dir}/time_state_scaled_val.pt')

# torch.save(macro_state, f'{save_dir}/macro_state_short.pt')
# torch.save(T_state, f'{save_dir}/T_state_short.pt')
# torch.save(time_state, f'{save_dir}/time_state_short.pt')
# torch.save(time_state_scaled, f'{save_dir}/time_state_scaled_short.pt')
# torch.save(macro_state_partial, f'../data/atoms_{N_atoms}/macro_state_partial_short.pt')



In [ ]:
T_state.shape

### Load partial sampling data 

In [ ]:
macro_state_partial_sampling = []
macro_state_sampling = []
delta_t_state_partial_sampling = []
step_partial_sampling = []

# for output_dir in [output_dir_short, output_dir_long]:
for output_dir in [output_dir_short]:
    for folder in tqdm(os.listdir(output_dir)):
        T = int(folder.split('_')[1])
        folder_path = os.path.join(output_dir, folder)

        macro_state_partial_sampling.append(np.load(os.path.join(folder_path, 'macro_vals_partial_sampling.npy')))
        macro_state_sampling.append(np.load(os.path.join(folder_path, 'macro_vals_sampling.npy')))
        delta_t_state_partial_sampling.append(np.load(os.path.join(folder_path, 'kmc_times_partial_sampling.npy')))
        step_partial_sampling.append(np.load(os.path.join(folder_path, 'step_partial_sampling.npy')))

macro_state_partial_sampling = np.stack(macro_state_partial_sampling, axis=0)
delta_t_state_partial_sampling = np.stack(delta_t_state_partial_sampling, axis=0)
macro_state_sampling = np.stack(macro_state_sampling, axis=0)
step_partial_sampling = np.stack(step_partial_sampling, axis=0)

In [ ]:
step_partial_sampling.shape

In [ ]:
macro_state = macro_state.detach().cpu().numpy()
macro_state_partial = macro_state_partial.detach().cpu().numpy()
T_state = T_state.detach().cpu().numpy()
time_state = time_state.detach().cpu().numpy()
time_state_scaled = time_state_scaled.detach().cpu().numpy()

In [ ]:
idx = 6
delta = macro_state_partial[idx] - macro_state[idx, :-1]
delta_partial = macro_state_partial_sampling[idx, ] - macro_state_sampling[idx, :-1]

fig = plt.figure(figsize=(40, 6))
for i in range(6):
    axes = fig.add_subplot(1, 6, i+1)
    
    # # Plot distribution using histplot with normalization
    sns.histplot(delta[:, i] / np.sqrt((time_state[idx, 1:] - time_state[idx, :-1])), 
                 ax=axes, kde=True, bins=50, alpha=0.5, color='skyblue', label='Full Sampling', stat='density')

    sns.histplot(delta_partial[:, i] / np.sqrt(delta_t_state_partial_sampling[idx]), 
                 ax=axes, kde=True, bins=100, alpha=0.5, color='tab:orange', label='Partial Sampling', stat='density')

    axes.set_title(f'Component {i}')
    axes.set_xlabel('Normalized Delta')
    axes.set_ylabel('Density')
    # axes.legend()

plt.tight_layout()
plt.show()

In [ ]:
idx = 7
delta = macro_state_partial[idx] - macro_state[idx, :-1]
delta_partial = macro_state_partial_sampling[idx, ] - macro_state_sampling[idx, :-1]

norm = np.linalg.norm(delta, axis=1)
norm_partial = np.linalg.norm(delta_partial, axis=1)

plt.plot(norm, 'ro', markersize=1, label='Full Sampling')
plt.plot(norm_partial, 'bo', markersize=1, label='Partial Sampling')
plt.show()

In [ ]:
delta_t_state_partial_sampling_scaled = delta_t_state_partial_sampling * scale_function(T_state[:, :-1])

In [ ]:
plt.plot(T_state[:, :-1], time_state_scaled[:, 1:] - time_state_scaled[:, :-1], 'ro', alpha=1, markersize=0.05)
plt.plot(T_state[:, :-1], delta_t_state_partial_sampling_scaled, 'bo', alpha=0.05, markersize=0.05)
plt.yscale('log')
plt.show()

In [ ]:
macro_state_partial_sampling = torch.tensor(macro_state_partial_sampling, dtype=torch.float32)
macro_state_sampling = torch.tensor(macro_state_sampling, dtype=torch.float32)
delta_t_state_partial_sampling_scaled = torch.tensor(delta_t_state_partial_sampling_scaled, dtype=torch.float32)

In [ ]:

 # ========= save partial data =========
# save the partial macro state and idx_partial
torch.save(macro_state_partial_sampling[train_idx], f'../data/atoms_{N_atoms}/macro_state_partial_sampling.pt')
torch.save(macro_state_sampling[train_idx], f'../data/atoms_{N_atoms}/macro_state_sampling.pt')
torch.save(delta_t_state_partial_sampling_scaled[train_idx], f'../data/atoms_{N_atoms}/delta_t_scaled_partial_sampling.pt')


In [ ]:
macro_state_partial_sampling.shape, macro_state_sampling.shape, delta_t_state_partial_sampling_scaled.shape

### Load partial sampling data for all temperatures

In [ ]:
folder = '../data/partial_sampling_atoms_8192'

# kmc_time = []
# macro_val_partial = []
# macro_val = []
# T_state = []
# step = []
# for T in [1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000]:
#     kmc_time.append(np.load(os.path.join(folder, f'kmc_times_T_{T}.npy')))
#     macro_val_partial.append(np.load(os.path.join(folder, f'macro_vals_partial_T_{T}.npy')))
#     macro_val.append(np.load(os.path.join(folder, f'macro_vals_T_{T}.npy')))
#     T_state.append(T * np.ones_like(kmc_time[-1]))
#     step.append(np.load(os.path.join(folder, f'step_T_{T}.npy')))

# T_state = np.concatenate(T_state, axis=0)
# kmc_time = np.concatenate(kmc_time, axis=0)
# kmc_time = kmc_time * scale_function(T_state)
# macro_val_partial = np.concatenate(macro_val_partial, axis=0)
# macro_val = np.concatenate(macro_val, axis=0)


# kmc_time.shape, macro_val_partial.shape, macro_val.shape, T_state.shape

In [ ]:
plt.plot(T_state, kmc_time, 'ro', markersize=0.1)
plt.yscale('log')
plt.show()

In [ ]:
N_atoms = 8192
kmc_time = torch.tensor(kmc_time, dtype=torch.float32)
macro_val_partial = torch.tensor(macro_val_partial, dtype=torch.float32)
macro_val = torch.tensor(macro_val, dtype=torch.float32)
T_state = torch.tensor(T_state, dtype=torch.int32)

torch.save(macro_val_partial, f'../data/atoms_{N_atoms}/macro_state_partial_sampling_tile.pt')
torch.save(macro_val, f'../data/atoms_{N_atoms}/macro_state_sampling_tile.pt')
torch.save(kmc_time, f'../data/atoms_{N_atoms}/delta_t_scaled_partial_sampling_tile.pt')
torch.save(T_state, f'../data/atoms_{N_atoms}/T_state_tile.pt')

In [ ]:
idx = np.where(T_state[:, 0] == T)[0][0]


In [ ]:
T = 2000
kmc_time = np.load(os.path.join(folder, f'kmc_times_T_{T}.npy'))
z1_train_partial = np.load(os.path.join(folder, f'z1_train_partial_T_{T}.npy'))
z0_train = np.load(os.path.join(folder, f'z0_train_T_{T}.npy'))
step = np.load(os.path.join(folder, f'step_T_{T}.npy'))

In [ ]:
np.mean(step)

In [ ]:
fig = plt.figure(figsize=(8, 6))
axes = fig.add_subplot(1, 1, 1)

sns.kdeplot(kmc_time, ax=axes, alpha=0.5, color='tab:orange', label='Partial Sampling')
# sns.kdeplot(time_state[idx, 1:] - time_state[idx, -1:], ax=axes, alpha=0.5, color='tab:orange', label='Partial Sampling')

axes.set_title(f'Component {i}')
axes.set_xlabel('Normalized Delta')
axes.set_ylabel('Density')
# axes.legend()

plt.tight_layout()
plt.show()

In [ ]:
plt.plot(time_state[1, 1:] - time_state[1, :-1], 'ro')
plt.show()

In [ ]:
(time_state[idx, 1:] - time_state[idx, -1:]).shape

In [ ]:
# idx = 6
idx = np.where(T_state[:, 0] == T)[0][0]

fig = plt.figure(figsize=(40, 6))
for i in range(6):
    axes = fig.add_subplot(1, 6, i+1)
    
    # Plot distribution using histplot with normalization
    sns.histplot((macro_state_partial[idx, :, i] - macro_state[idx, :-1, i]) / np.sqrt((time_state[idx, 1:] - time_state[idx, :-1])),
                 ax=axes, kde=True, bins=50, alpha=0.5, color='skyblue', label='Full Sampling', stat='density')

    sns.histplot((z1_train_partial[:, i] - z0_train[:, i]) / np.sqrt(kmc_time[:]),
                 ax=axes, kde=True, bins=50, alpha=0.5, color='tab:orange', label='Partial Sampling', stat='density')

    axes.set_title(f'Component {i}')
    axes.set_xlabel('Normalized Delta')
    axes.set_ylabel('Density')
    # axes.legend()

plt.tight_layout()
plt.show()

In [ ]:
idx = 6
idx = np.where(T_state[:, 0] == T)[0][0]

fig = plt.figure(figsize=(40, 6))
for i in range(6):
    axes = fig.add_subplot(1, 6, i+1)
    
    # Plot distribution using histplot with normalization
    sns.histplot(macro_state[idx, :-1, i],
                 ax=axes, kde=True, bins=50, alpha=0.5, color='skyblue', label='Full Sampling', stat='density')

    # sns.histplot(z0_train[:, i],
    #              ax=axes, kde=True, bins=50, alpha=0.5, color='tab:orange', label='Partial Sampling', stat='density')

    axes.set_title(f'Component {i}')
    axes.set_xlabel('Normalized Delta')
    axes.set_ylabel('Density')
    # axes.legend()

plt.tight_layout()
plt.show()

In [ ]:
T = 2000
fig = plt.figure(figsize=(40, 6))
for i in range(6):
    axes = fig.add_subplot(1, 6, i+1)
    for j in [6]:
        axes.plot(time_state[j], macro_state[j, :, i])
    axes.set_xlabel('Step')
    axes.set_ylabel(f'Delta_{i}')
    axes.set_ylim(-3.5, 3.5)
plt.title(f'T = {T} K')
plt.tight_layout()

In [ ]:
torch.mean(macro_state[idx], axis=0)
